# **ONSAGER TESTING**

## **Installs**

### Firedrake

In [7]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2026-02-06 18:08:52--  https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4775 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2026-02-06 18:08:52 (57.4 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4775/4775]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ set +x
























#########################################################

### Gmsh

In [8]:
try:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh"
    !bash "/tmp/gmsh-install.sh"
    import gmsh  # noqa: F401
except:
    import gmsh  # noqa: F401

--2026-02-06 18:08:53--  https://fem-on-colab.github.io/releases/gmsh-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3497 (3.4K) [application/x-sh]
Saving to: ‘/tmp/gmsh-install.sh’

/tmp/gmsh-install.s 100%[===================>]   3.42K  --.-KB/s    in 0s      

2026-02-06 18:08:53 (43.4 MB/s) - ‘/tmp/gmsh-install.sh’ saved [3497/3497]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ GMSH_INSTALLED=/usr/local/share/fem-on-colab/gmsh.installed
+ [[ ! -f /usr/local/share/fem-on-colab/gmsh.installed ]]
+ set +x
























################################################################################
#     This installation

## **Code**

### Mesh

In [9]:
def rect_with_hole_mesh(r=0.5, left=1.0, right=4.0, updown=1.0, h=0.2, obstacle="circle"):
    '''
    r = hole radius
    left = space to left of hole centre
    right = space to right of hole centre
    updown = space above / below hole centre
    h = (target) mesh size
    '''
    gmsh.initialize()
    gmsh.model.add("rect_with_hole")

    # Derived rectangle bounds
    xmin = -left
    xmax = right
    ymin = -updown
    ymax = updown


    # -------------------
    # Rectangle
    # -------------------
    p1 = gmsh.model.geo.addPoint(xmin, ymin, 0, h)
    p2 = gmsh.model.geo.addPoint(xmax, ymin, 0, h)
    p3 = gmsh.model.geo.addPoint(xmax, ymax, 0, h)
    p4 = gmsh.model.geo.addPoint(xmin, ymax, 0, h)

    l_bottom = gmsh.model.geo.addLine(p1, p2)
    l_right  = gmsh.model.geo.addLine(p2, p3)
    l_top    = gmsh.model.geo.addLine(p3, p4)
    l_left   = gmsh.model.geo.addLine(p4, p1)

    rect_loop = gmsh.model.geo.addCurveLoop(
        [l_bottom, l_right, l_top, l_left]
    )


    # -------------------
    # Hole (centred at origin)
    # -------------------
    if obstacle=="circle":
        c  = gmsh.model.geo.addPoint(0, 0, 0, h)

        cp = gmsh.model.geo.addPoint( r, 0, 0, h)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h)

        a1 = gmsh.model.geo.addCircleArc(cp, c, ct)
        a2 = gmsh.model.geo.addCircleArc(ct, c, cn)
        a3 = gmsh.model.geo.addCircleArc(cn, c, cb)
        a4 = gmsh.model.geo.addCircleArc(cb, c, cp)
    elif obstacle=="semicircle":
        c  = gmsh.model.geo.addPoint(0, 0, 0, h)

        cp = gmsh.model.geo.addPoint(0,  0, 0, h)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addCircleArc(ct, c, cn)
        a3 = gmsh.model.geo.addCircleArc(cn, c, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)
    elif obstacle=="diamond":
        cp = gmsh.model.geo.addPoint( r, 0, 0, h)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addLine(ct, cn)
        a3 = gmsh.model.geo.addLine(cn, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)
    elif obstacle=="wedge":
        cp = gmsh.model.geo.addPoint( 0, 0, 0, h)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addLine(ct, cn)
        a3 = gmsh.model.geo.addLine(cn, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)
    elif obstacle=="square":
        cp = gmsh.model.geo.addPoint( r,  r, 0, h)
        cn = gmsh.model.geo.addPoint(-r, -r, 0, h)
        ct = gmsh.model.geo.addPoint(-r,  r, 0, h)
        cb = gmsh.model.geo.addPoint(r,  -r, 0, h)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addLine(ct, cn)
        a3 = gmsh.model.geo.addLine(cn, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)

    hole_loop = gmsh.model.geo.addCurveLoop([a1, a2, a3, a4])


    # -------------------
    # Surface with hole
    # -------------------
    surface = gmsh.model.geo.addPlaneSurface([rect_loop, hole_loop])

    gmsh.model.geo.synchronize()


    # -------------------
    # Physical groups (THIS is the important bit)
    # -------------------
    gmsh.model.addPhysicalGroup(1, [l_left], tag=1)
    gmsh.model.setPhysicalName(1, 1, "inlet")

    gmsh.model.addPhysicalGroup(1, [l_right], tag=2)
    gmsh.model.setPhysicalName(1, 2, "outlet")

    gmsh.model.addPhysicalGroup(1, [l_top, l_bottom], tag=3)
    gmsh.model.setPhysicalName(1, 3, "wall")

    gmsh.model.addPhysicalGroup(1, [a1, a2, a3, a4], tag=4)
    gmsh.model.setPhysicalName(1, 4, "hole")

    gmsh.model.addPhysicalGroup(2, [surface], tag=5)


    # -------------------
    # Mesh
    # -------------------
    gmsh.model.mesh.generate(2)
    gmsh.write("rect_with_hole.msh")
    gmsh.finalize()

    return Mesh("rect_with_hole.msh")

### Euler (Viscosity solution)

In [13]:
def euler(h=0.2, sigma=10.0, Re=1.0, dt=2**-5, T=2**-3, save=True, obstacle="semicircle", skew_symmetric=False):
    # -----------
    #    Setup
    # -----------
    # Mesh
    mesh = rect_with_hole_mesh(h=h, obstacle=obstacle)
    n = FacetNormal(mesh)

    # Normalise parameters
    sigma_c = Constant(sigma)
    dt_c = Constant(dt)

    # Spaces
    V = FunctionSpace(mesh, "N1curl", 1)
    Q = FunctionSpace(mesh, "CG", 1)
    W = V * Q

    # Unknowns
    w = Function(W)
    u, p = split(w)
    u_sol, p_sol = w.subfunctions
    u_sol.rename("velocity")
    p_sol.rename("pressure")

    # Old velocity
    u_old = Function(V)

    # Tests
    v, q = TestFunctions(W)

    # Solver parameters
    sp = {
        "snes_monitor" : None,
        "snes_converged_reason" : None,
        # "snes_atol" : 1e-10,
        # "snes_rtol" : 1e-10,
    }


    # --------------------------
    #    Initial Stokes solve
    # --------------------------
    # Operators
    def jump_normal_int(w):
        return inner(w('+') - w('-'), n('+'))
    def jump_int(w):
        return jump_normal_int(w) * outer(n('+'), n('+'))
    def jump_normal_ext(w, inlet=False):
        if inlet:
            return inner(w, n) + 1
        else:
            return inner(w, n)
    def jump_ext(w, inlet=0):
        return jump_normal_ext(w, inlet) * outer(n, n)

    # Viscous term
    viscous_int = sigma_c / 4 * inner(jump_normal_int(u), jump_normal_int(v)) * dS
    viscous_ext = sigma_c / 4 * (
        inner(jump_normal_ext(u, inlet=True), jump_normal_ext(v)) * ds(1)  # Inlet
      + inner(jump_normal_ext(u), jump_normal_ext(v)) * (ds(3) + ds(4))  # Walls + Obstacle
    ) - p * inner(v, n) * ds(2)  # Outlet

    # Pressure term
    pressure_bulk = inner(grad(p), v) * dx

    # Incompressibility term
    incomp_bulk = inner(u, grad(q)) * dx
    incomp_ext = q * ds(1) - inner(u, n) * q * ds(2)

    # Residual
    F = (
        viscous_int + viscous_ext
      + pressure_bulk
      + incomp_bulk + incomp_ext
    )

    # Boundary conditions
    bc = DirichletBC(W.sub(0), as_vector([0.0, 0.0]), 1)

    # Solve
    print(GREEN % f"Setting up ICs for t = 0 (uniform Re):")
    solve(F == 0, w, bcs=bc, solver_parameters=sp)


    # ----------------------------
    #    Modified Stokes update
    # ----------------------------
    # Viscous term
    viscous_int = sigma_c / 4 * abs(jump_normal_int(u)) * inner(jump_normal_int(u), jump_normal_int(v)) * dS

    # Residual
    F = (
        viscous_int + viscous_ext
      + pressure_bulk
      + incomp_bulk + incomp_ext
    )

    # PVD setup
    if save:
        !rm -rf velocity*
        outfile = VTKFile("velocity.pvd")

    # Solve
    print(GREEN % f"Setting up ICs for t = 0 (non-uniform Re):")
    solve(F == 0, w, bcs=bc, solver_parameters=sp)
    if save: outfile.write(u_sol, p_sol)
    u_old.assign(u_sol)


    # ------------------------------
    #    Full Navier-Stokes solve
    # ------------------------------
    # Operators
    def avg_ext(op, u, inlet=0):
        if inlet==0:
            u_ext = u - inner(u, n) * n
        else:
            u_ext = u - (inner(u, n) + inlet) * n
        return 1/2 * (op(u) + op(u_ext))
    def avg_u_outer_ext(inlet=0):
        return avg_ext(lambda u : outer(u, u), u, inlet)
    def avg_u_inner_ext(inlet=0):
        return avg_ext(lambda u : inner(u, u), u, inlet)

    # Transient term
    transient_bulk = 1 / dt_c * inner((u - u_old), v) * dx

    # Advective term
    if skew_symmetric:
        advective_int = (
            inner(outer(avg(u), avg(u)) + 1/12 * outer(u('+')-u('-'), u('+')-u('-')), sym(jump_int(v)))
          + 1/2 * inner(inner(avg(u), avg(u)) + 1/12 * inner(u('+')-u('-'), u('+')-u('-')), jump_normal_int(v))
        ) * dS
    else:
        advective_int = (
            inner(avg(outer(u, u)), sym(jump_int(v)))
          + 1/2 * inner(avg(inner(u, u)), jump_normal_int(v))
        ) * dS

    if skew_symmetric:
        raise("TO-DO...")
    else:
        advective_ext = (
            (
                inner(avg_u_outer_ext(inlet=1), sym(jump_ext(v)))
              - inner(u, v)
              + 1/2 * inner(avg_u_inner_ext(inlet=1), jump_normal_ext(v))
            ) * ds(1)  # Inlet
          + (
                inner(avg_u_outer_ext(), sym(jump_ext(v)))
              + 1/2 * inner(avg_u_inner_ext(), jump_normal_ext(v))
            ) * (ds(3) + ds(4))  # Walls + Obstacle
          + (
                inner(u, n) * inner(u, v)
            ) * ds(2)  # Outlet
        )

    # Residual
    F += (
        transient_bulk
      + advective_int + advective_ext
    )

    # Time loop
    t = 0.0
    step = 0
    while step < round(T/dt):
        t += dt
        step += 1
        print(GREEN % f"Solving for t = {t:.4e}:")
        solve(F == 0, w, bcs=bc, solver_parameters=sp)
        if save: outfile.write(u_sol, p_sol)
        u_old.assign(u_sol)

In [11]:
def download():
    import zipfile
    import glob
    import os
    from google.colab import files

    # Zip the .pvd and all .vtu files
    zip_name = "velocity_data.zip"
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        if os.path.exists("velocity.pvd"):
            zipf.write("velocity.pvd")

        # The .vtu files are stored in a folder called 'velocity'
        if os.path.isdir("velocity"):
            for root, _, files_list in os.walk("velocity"):
                for file in files_list:
                    if file.endswith(".vtu"):
                        file_path = os.path.join(root, file)
                        zipf.write(file_path, arcname=file_path)

    files.download(zip_name)
    print(BLUE % f"Download complete!")

In [1]:
euler(h=0.02, sigma=20.0, dt=0.2, T=20.0)
download()

NameError: name 'download' is not defined